### Configure API

In [22]:
import requests
import datetime
import time
import pandas as pd

In [3]:
session =requests.Session()

In [13]:
#configure payload
prox = "20" #proximity in metres
app_id = #Look up lambda'
app_code = #Look up lambda
firstCord = '-28.14928242,153.4796733'
url = "https://traffic.api.here.com/traffic/6.2/flow.json?app_id=" + app_id + "&app_code=" + app_code
url +="&prox="+firstCord+","+prox+"&responseattributes=sh,fc"

In [9]:
#send request
starttime = time.time()
urlsession = requests.session()
response = requests.get(url, timeout=600)    
response = response.content

ConnectionError: HTTPSConnectionPool(host='traffic.api.here.com', port=443): Max retries exceeded with url: /traffic/6.2/flow.json?app_id=Q56YtQZX205BCqVWB4UT&app_code=RkLmGtCor_WSh79Xg4egzA&prox=-28.14928242,153.4796733,20&responseattributes=sh,fc (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000000002A34B70>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [11]:
session =requests.Session()
session.proxies ={
    'http':'localhost:3128',
    'https':'localhost:3128'
}

### Send Request to HERE 

1. Config payload and response

In [15]:
response = session.get(url)

In [16]:
response.headers['Content-Type']

'application/json;charset=utf-8'

In [19]:
flowInfo = response.json()

#### Initialise dictionaries

In [23]:
dfcols = ['id','name','avSpeed','jamF','cords']
dfHere = pd.DataFrame(columns = dfcols)

In [24]:
flowInfo

{'RWS': [{'RW': [{'FIS': [{'FI': [{'TMC': {'PC': 25047,
          'DE': 'Stewart Road',
          'QD': '-',
          'LE': 2.51785},
         'SHP': [{'value': ['-28.14909,153.4798 -28.14903,153.4798 -28.14867,153.47978 -28.14825,153.47979 -28.14783,153.47982 -28.14718,153.47987 -28.14671,153.47987 '],
           'FC': 1},
          {'value': ['-28.14671,153.47987 -28.14631,153.47985 -28.14609,153.47982 '],
           'FC': 1},
          {'value': ['-28.14609,153.47982 -28.14591,153.47979 '], 'FC': 1},
          {'value': ['-28.14591,153.47979 -28.14578,153.47976 '], 'FC': 1},
          {'value': ['-28.16115,153.49605 -28.16088,153.49572 -28.16058,153.49532 '],
           'FC': 1},
          {'value': ['-28.16058,153.49532 -28.16033,153.49498 -28.16,153.4945 -28.15963,153.49392 -28.15918,153.49312 -28.15897,153.49273 -28.15877,153.49232 -28.15849,153.49172 -28.15827,153.49122 '],
           'FC': 1},
          {'value': ['-28.15827,153.49122 -28.15805,153.49072 -28.15758,153.48972 -2

In [33]:
flowInfo["RWS"]["FIS"]

TypeError: list indices must be integers or slices, not str

In [44]:
r = flowInfo

dict_keys(['RWS', 'MAP_VERSION', 'CREATED_TIMESTAMP', 'VERSION', 'UNITS'])

In [42]:
for el1 in r['RWS']:
    for el2 in el1['RW']:
        masId = el2['mid']
        masRd = str(el2['DE'])
        totSp= 0
        totJm = 0
        print("---------")
        print(masRd + " -MajorRd") #name
        for el3 in el2['FIS']: #Road level
            for el4 in el3['FI']: #flow information extract here at link level
                linRd = el4['TMC']
                print(linRd['DE'] +" - linklvl") #name
                linRd = linRd['DE']
                cordsSwap2=[]
                for el5 in el4['SHP']: #get shape file
                    #swap patter lat,lon to lon,lat       
                    x=[]
                    tmpStr = str(el5['value']).replace(" -2","|-2").replace("[u'","").replace("']","")
                    x=tmpStr.split("|")
                    cordsSwap=""
                    for index in range(len(x)):
                        lat, lon = x[index].split(",")
                        cordsSwap += ",["+lon+","+lat+"]" #we need to check this part!!
                    cordsSwap2.append(cordsSwap)
                for el6 in el4['CF']:#get shape file
                    linkSpeed = int(el6['SU']) #Speed
                    linkJam = int(el6['JF']) #JamFactor
                    print(linkSpeed)
                    print(linkJam)
                print(linRd)
                cords =""
                for j in range(len(cordsSwap2)):
                    cords += cordsSwap2[j]+","
                cords = cords[:-1]
                print(cords)
                dfHere.loc[len(dfHere)] = [masRd, linRd, linkSpeed,linkJam,cords]
                #dfHere.loc[len(dfHere)] = [masRd, linRd, linkSpeed,linkJam,cords]

---------
Tugun Bypass -MajorRd
Stewart Road - linklvl
---------
Tugun Bypass -MajorRd
Stewart Road - linklvl
---------
Currumbin Creek Road -MajorRd
Stewart Road - linklvl
---------
Currumbin Creek Road -MajorRd
Stewart Road - linklvl
---------
Gold Coast Highway -MajorRd
Stewart Road - linklvl
---------
Gold Coast Highway -MajorRd
Toolona Street - linklvl
---------
Pacific Motorway -MajorRd
Stewart Road - linklvl
---------
Pacific Motorway -MajorRd
Stewart Road - linklvl
---------
Stewart Road -MajorRd
Currumbin Creek Road - linklvl
---------
Stewart Road -MajorRd
Gold Coast Highway - linklvl
---------
Currumbin Creek Road -MajorRd
Stewart Road - linklvl
---------
Currumbin Creek Road -MajorRd
Stewart Road - linklvl


### Json debug

In [43]:
for el1 in r['RWS']:
    for el2 in el1['RW']:
        for el3 in el2['FIS']: #Road level
            for el4 in el3['FI']: #flow information extract here at link level
                linRd = el4['TMC']
                print(linRd['DE'] +" - linklvl") #name
                linRd = linRd['DE']
                cordsSwap2=[]
                break
                for el5 in el4['SHP']: #get shape file
                    #swap patter lat,lon to lon,lat       
                    x=[]
                    tmpStr = str(el5['value']).replace(" -2","|-2").replace("[u'","").replace("']","")
                    x=tmpStr.split("|")
                    cordsSwap=""
                    for index in range(len(x)):
                        lat, lon = x[index].split(",")
                        cordsSwap += ",["+lon+","+lat+"]" #we need to check this part!!
                    cordsSwap2.append(cordsSwap)
                for el6 in el4['CF']:#get shape file
                    linkSpeed = int(el6['SU']) #Speed
                    linkJam = int(el6['JF']) #JamFactor
                    print(linkSpeed)
                    print(linkJam)
                print(linRd)
                cords =""
                for j in range(len(cordsSwap2)):
                    cords += cordsSwap2[j]+","
                cords = cords[:-1]
                print(cords)
                dfHere.loc[len(dfHere)] = [masRd, linRd, linkSpeed,linkJam,cords]
                #dfHere.loc[len(dfHere)] = [masRd, linRd, linkSpeed,linkJam,cords]

Stewart Road - linklvl
Stewart Road - linklvl
Stewart Road - linklvl
Stewart Road - linklvl
Stewart Road - linklvl
Toolona Street - linklvl
Stewart Road - linklvl
Stewart Road - linklvl
Currumbin Creek Road - linklvl
Gold Coast Highway - linklvl
Stewart Road - linklvl
Stewart Road - linklvl


In [46]:
r.keys()

dict_keys(['RWS', 'MAP_VERSION', 'CREATED_TIMESTAMP', 'VERSION', 'UNITS'])

In [71]:
i=0
for el1 in r['RWS']:
    i+=1
    for el2 in el1['RW']:
        #print(el2['DE'] +" - linklvl") #name
        linRd = el2['DE']
        #print(i)
        #print(el2.keys())

Tugun Bypass - linklvl
Tugun Bypass - linklvl
Currumbin Creek Road - linklvl
Currumbin Creek Road - linklvl
Gold Coast Highway - linklvl
Gold Coast Highway - linklvl
Pacific Motorway - linklvl
Pacific Motorway - linklvl
Stewart Road - linklvl
Stewart Road - linklvl
Currumbin Creek Road - linklvl
Currumbin Creek Road - linklvl


In [96]:
i=0
for el1 in r['RWS']:
    i+=1
    for el2 in el1['RW']:
            for el3 in el2['FIS']: #Road level
                for el4 in el3['FI']: #flow information extract here at link level
                    linRd = el4['TMC']
                    linRd = linRd['DE']
                    print(linRd)
                    for el5 in el4['SHP']: #get shape file
                        x=[]
                        cordsSwap2=[]
                        tmpStr = str(el5['value']).replace(" -2","|-2").replace("[u'","").replace("']","")
                        x=tmpStr.split("|")
                        cordsSwap=""
                        for index in range(len(x)):
                            lat, lon = x[index].split(",")
                            cordsSwap += "["+lon+","+lat+"]" #we need to check this part!!
                        cordsSwap2.append(cordsSwap)
                        print('ss')
                        print(cordsSwap2)
                        break
                    break
                    for el6 in el4['CF']:#get shape 
                        print('aaa')
                    #for el5 in el4['TMC']: #link info
                        #linRd = el4['TMC']
                        #linRd = el5['DE']
                        #print(linRd)
                        #print(el5.keys())
                        #linkRd = el5['DE']
                        #print(el5['DE'])

Stewart Road
ss
["[153.4798,['-28.14909][153.4798,-28.14903][153.47978,-28.14867][153.47979,-28.14825][153.47982,-28.14783][153.47987,-28.14718][153.47987 ,-28.14671]"]
Stewart Road
ss
["[153.47998,['-28.14574][153.48001 ,-28.14587]"]
Stewart Road
ss
["[153.47109,['-28.1467][153.47114,-28.14658][153.47118,-28.1465][153.47125,-28.14637][153.47129,-28.14631][153.47136 ,-28.14624]"]
Stewart Road
ss
["[153.47307,['-28.13834][153.47298,-28.13839][153.47289,-28.13845][153.47282,-28.1385][153.47275,-28.13856][153.47269,-28.13863][153.47264 ,-28.1387]"]
Stewart Road
ss
["[153.49323,['-28.14311][153.4931,-28.14302][153.49294,-28.14291][153.49283,-28.14283][153.49268,-28.14271][153.49259,-28.14263][153.49248 ,-28.14252]"]
Toolona Street
ss
["[153.49231,['-28.14211][153.49245 ,-28.1423]"]
Stewart Road
ss
["[153.47976,['-28.14578][153.47972 ,-28.14559]"]
Stewart Road
ss
["[153.47832,['-28.14273][153.47851,-28.14293][153.47871,-28.14315][153.47892,-28.14341][153.4791,-28.14366][153.47931 ,-28.144]"

In [258]:
def changeCoordsStr(latLong):
    """
    takes dictionary - change format from [lat,long] to [long,lat]
    {'value': ['-28.14909,153.4798 -28.14903,153.4798 -28.14867,153.47978 -28.14825,153.47979 -28.14783,153.47982 -28.14718,153.47987 -28.14671,153.47987 '], 'FC': 1} to
    '[153.4798,-28.14909],[153.4798,-28.14903],[153.47978,-28.14867],[153.47979,-28.14825],[153.47982,-28.14783],[153.47987,-28.14718],[153.47987,-28.14671]''   
    """

    tmpStr = str(latLong.get('value')).replace("'","")
    tmpStr = tmpStr.replace(" -2","|-2").replace("[","").replace("]","").strip()

    tmpStr= tmpStr.split("|")
    cordsSwap="["
    for index in range(len(tmpStr)):
        lat, lon = tmpStr[index].split(",")
        cordsSwap += "["+lon+","+lat+"],"
    cordsSwap = cordsSwap[:-1]+"]" #remove the last character ',' and close ']'
    return cordsSwap 

In [259]:
latStr ='[-28.14909,153.4798],[-28.14903,153.4798],[-28.14867,153.47978],[-28.14825,153.47979],[-28.14783,153.47982],[-28.14718,153.47987],[-28.14671,153.47987]'
latStr = {'value': ['-28.14909,153.4798 -28.14903,153.4798 -28.14867,153.47978 -28.14825,153.47979 -28.14783,153.47982 -28.14718,153.47987 -28.14671,153.47987 '], 'FC': 1}
changeCoordsStr(latStr) 

'[[153.4798,-28.14909],[153.4798,-28.14903],[153.47978,-28.14867],[153.47979,-28.14825],[153.47982,-28.14783],[153.47987,-28.14718],[153.47987,-28.14671]]'

In [260]:
i=0
dfcols = ['id','name','avSpeed','jamF','cords']
dfHere = pd.DataFrame(columns = dfcols)

for el1 in r['RWS']:
    for el2 in el1['RW']:
            for el3 in el2['FIS']: #Road level
                for el4 in el3['FI']: #flow information extract here at link level
                    linRd = el4['TMC'].get('DE')
                    #linRd = linRd['DE']
                    print(linRd)
                    flowInfoSpeed = el4['CF'][0].get('SU') #Speed (based on UNITS) not capped by speed limit
                    flowInfoJam = el4['CF'][0].get('JF') # The number between 0.0 and 10.0 indicating the expected quality of travel. When there is a road closure, the Jam Factor will be 10. As the number approaches 10.0 the quality of travel is getting worse. -1.0 indicates that a Jam Factor could not be calculated
                    flowInfoCon =  el4['CF'][0].get('CN') #Confidence, an indication of how the speed was determined. -1.0 road closed. 1.0=100% 0.7-100% Historical Usually a value between .7 and 1.0
                    for el5 in el4['SHP']: #get shape file
                        i+=1
                        cordStr = changeCoordsStr(el5)
                        dfHere.loc[len(dfHere)] = [i, linRd, flowInfoSpeed,flowInfoJam,cordStr]


Stewart Road
Stewart Road
Cobaki Parkway
Stewart Road
Pacific Motorway
Stewart Road
Stewart Road
Toolona Street
Stewart Road
Currumbin Creek Road
Stewart Road
Currumbin Creek Road
Pacific Motorway
Gold Coast Highway
Gold Coast Highway
Pacific Motorway
Currumbin Creek Road
Stewart Road
Galleon Way
Stewart Road


In [263]:
dfHere.iloc[2]

id                                                     3
name                                        Stewart Road
avSpeed                                            95.34
jamF                                             0.54742
cords      [[153.47982,-28.14609],[153.47979,-28.14591]]
Name: 2, dtype: object

### Output into path json file for extract